<a href="https://colab.research.google.com/github/moyen-ahmed/Deep-learning/blob/main/model_desgin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install einops

# ---- core libs ----
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
import tqdm, time, copy
import heapq
import datetime
import glob
import math, random
import pathlib
import itertools
import matplotlib.pyplot as plt

from functools import partial

# ---- Keras / TF high-level APIs (use ONLY tensorflow.keras) ----
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow.keras.utils import *
from tensorflow.keras.regularizers import *
# from tensorflow.keras.layers import Lambda   # use Lambda from tf.keras, not keras

# ---- sklearn ----
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# ---- einops ----
import einops
from einops import *

# ---- random_time replacement (no pip package needed) ----
import types as _types
import time as _time

class RandomTimeModule:
    """
    Simple drop-in replacement so `random_time` exists.
    Adjust functions if your later code needs something specific.
    """
    @staticmethod
    def random_delay(min_seconds=0.0, max_seconds=1.0):
        return random.uniform(min_seconds, max_seconds)

    @staticmethod
    def now():
        return _time.time()

# You can use: random_time.random_delay(...), random_time.now()
random_time = RandomTimeModule()

# ---- low-level TF ops (only the valid ones) ----
from tensorflow import (
    abs, cast, clip_by_value, concat, convert_to_tensor,
    expand_dims, gather, gather_nd, linspace, map_fn,
    norm, pad, repeat, reshape, shape, split,
    squeeze, stack, tensor_scatter_nd_update, unstack, zeros,
)


In [ ]:
!pip install einops

import keras
from keras import layers, Model   # everything from keras, not tensorflow.keras

kr = 2                     # Example hyper-parameter (could be used later, e.g. to scale channels)

image_size = 32            # Each input image is 32x32 pixels

xput = keras.Input(        # Keras Input layer: defines the shape of one image
    shape=(image_size,     #   height  = 32
           image_size,     #   width   = 32
           3)              #   depth   = 3 (RGB channels)
)

d1 = xput                  # For now, d1 is just the same tensor as xput
                           # In a real model, you would usually do:
                           # d1 = layers.Conv2D(...)(xput), etc.


# Example: if we add a Conv2D layer, the number of parameters P is:
#   P = (N * M * D + 1) * K
# where:
#   N, M = kernel height and width
#   D    = number of input channels (depth)
#   K    = number of filters (output channels)
#   +1   = one bias term for each filter
#
# So each filter has N * M * D weights + 1 bias,
# and there are K filters in total.

# Example Conv2D layer so the formula has context:
# N = 3, M = 3, D = 3 (RGB), K = 64
# Number of parameters = (3*3*3 + 1) * 64 = (27 + 1) * 64 = 1792
# conv = layers.Conv2D(64, (3, 3), padding="same")(d1)


class_model = Model(       # Build the Keras Model object
    inputs=xput,           # model input tensor
    outputs=d1,            # model output tensor
    name="class_model"
)                          # currently an identity model (output = input)

class_model.summary()      # prints layer types, output shapes, and parameter counts


# Now for py tourch


In [24]:
# Install helper libraries to show model summaries.
# torchinfo (newer) and torchsummary (older, similar idea).
!pip install torchinfo
!pip install torchsummary

# Import the neural-network module from PyTorch.
# nn contains building blocks like Linear, Conv2d, ReLU, etc.
import torch.nn as nn

# Import the summary function from torchinfo.
# This will print a table of layers, output shapes, and parameter counts.
from torchinfo import summary


# ===============================
# 1. Define your model
# ===============================

# All custom models in PyTorch inherit from nn.Module.
class SimpleModel(nn.Module):
    # Constructor: here you define and create your layers.
    def __init__(self):
        # Call the parent class (nn.Module) constructor so that internal
        # PyTorch bookkeeping (like parameter registration) is set up correctly.
        super(SimpleModel, self).__init__()

        # At the moment this model is "empty": it has no layers.
        # It will simply pass its input directly to the output.
        # You could add layers here, for example:
        #
        # self.conv1 = nn.Conv2d(
        #     in_channels=3,    # input has 3 channels (RGB image)
        #     out_channels=16,  # produce 16 feature maps
        #     kernel_size=3,    # 3x3 convolution kernel
        #     padding=1         # keep H,W the same size
        # )
        #
        # self.relu = nn.ReLU()
        #
        # self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    # This method defines how the input tensor flows through the model.
    # x is a batch of inputs (e.g. images).
    def forward(self, x):
        # For this SimpleModel, we do not change x at all.
        # In a real model, you would apply layers, for example:
        #
        # x = self.conv1(x)
        # x = self.relu(x)
        # x = self.pool(x)
        #
        # and then return the processed tensor.
        #
        # Here, we return x directly: this is an identity mapping.
        return x


# ===============================
# 2. Initialize the model
# ===============================

# Create an instance of your SimpleModel class.
# This allocates all parameters for the layers you defined in __init__.
model = SimpleModel()


# ===============================
# 3. Print model summary
# ===============================

# Use torchinfo.summary to print a readable summary of the model.
# The second argument is the expected input size (excluding batch dimension):
#   (3, 32, 32) means:
#       3   = number of channels  (e.g., RGB)
#       32  = height  in pixels
#       32  = width   in pixels
#
# torchinfo will:
#   • simulate a forward pass with a dummy input of shape (1, 3, 32, 32)
#   • list each layer
#   • show output shape for each layer
#   • count trainable and non-trainable parameters
summary(model, (3, 32, 32))


Layer (type:depth-idx)                   Output Shape              Param #
SimpleModel                              [3, 32, 32]               --
Total params: 0
Trainable params: 0
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0
Input size (MB): 0.01
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [1]:
from tensorflow.keras import layers, Model   # Import Keras layers and Model class

image_size = 32                              # We will work with 32x32 pixel images

# 1) Define the input tensor for the model
inputs = layers.Input(                       # Keras Input layer: starting point of the model graph
    shape=(image_size,                       #   height  = 32
           image_size,                       #   width   = 32
           3)                                #   depth   = 3 (RGB channels)
)                                            # Shape is (None, 32, 32, 3); None = batch size


# 2) First “stage” of the model
d1 = inputs                                  # d1 is just another name for the input tensor.
                                             # No operation is done yet; it is still (None, 32, 32, 3).


# 3) Apply a convolution layer to d1
d1 = layers.Conv2D(                          # Create a 2-D convolution layer
        10,                                  #   10 filters → output will have 10 channels
        kernel_size=(2, 2),                  #   each filter is 2x2 in spatial size
        padding='same'                       #   "same" padding → output H,W stay 32x32
     )(d1)                                   # Immediately apply this layer to tensor d1.
                                             # New shape becomes (None, 32, 32, 10).

#   Number of parameters in this Conv2D layer:
#   Formula: P = (N * M * D + 1) * K
#       N, M = kernel height and width = 2, 2
#       D    = number of input channels   = 3
#       K    = number of filters          = 10
#   P = (2 * 2 * 3 + 1) * 10 = (12 + 1) * 10 = 130 parameters


# 4) Build the Keras model object
class_model = Model(                         # Create a Model that connects input → output
    inputs=inputs,                           #   model input tensor (32x32x3 images)
    outputs=d1                               #   model output tensor (feature maps 32x32x10)
)                                            # This model now consists of one Conv2D layer.


# 5) Print a summary of the model architecture
class_model.summary()                        # Shows layer types, output shapes, and parameter counts


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 10)     │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 130 (520.00 B)

 Trainable params: 130 (520.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
# We assume torchinfo is already installed:
!pip install torchinfo

import torch.nn as nn
from torchinfo import summary


# ===============================
# 1. Define your model
# ===============================

class ConvModel(nn.Module):
    """
    A simple convolutional neural network with:
    - one Conv2d layer (10 filters, 2x2 kernel, 'same' padding)
    """

    def __init__(self):
        # Call the parent constructor to set up nn.Module internals
        super(ConvModel, self).__init__()

        # Define a convolutional layer with 10 filters, 2x2 kernel size, and 'same' padding.
        # Arguments:
        #   in_channels  = 3   (input image has 3 channels: RGB)
        #   out_channels = 10  (number of filters; output will have 10 feature maps)
        #   kernel_size  = (2, 2)  (filter height and width)
        #   padding      = 'same'  (output spatial size stays the same: 32x32 -> 32x32)
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=10,
            kernel_size=(2, 2),
            padding='same'
        )

        # Number of parameters in this conv layer:
        # Formula: P = (N * M * D + 1) * K
        #   N, M = kernel height and width = 2, 2
        #   D    = number of input channels   = 3
        #   K    = number of filters          = 10
        # So:
        #   P = (2 * 2 * 3 + 1) * 10 = (12 + 1) * 10 = 130 parameters (weights + biases)

    def forward(self, x):
        """
        Defines how the input tensor x flows through the network.

        x shape before conv1: (batch_size, 3, 32, 32)
        x shape after conv1:  (batch_size, 10, 32, 32)  # padding='same'
        """
        # Apply the convolutional layer to the input tensor x
        x = self.conv1(x)

        # Return the result (feature maps)
        return x


# ===============================
# 2. Initialize the model
# ===============================

# Create an instance of ConvModel, which allocates conv1 parameters.
model = ConvModel()


# ===============================
# 3. Print model summary
# ===============================

# Use torchinfo.summary to display the architecture.
# input_size is the shape of a single input sample (without batch size):
#   (3, 32, 32) = 3 channels, 32x32 pixels.
#
# torchinfo will internally create a dummy input of shape (1, 3, 32, 32)
# and run it through the model to gather shapes and parameter counts.
summary(
    model,
    input_size=(3, 32, 32)   # (channels, height, width)
)


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/conv.py:543: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /pytorch/aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv2d(


Layer (type:depth-idx)                   Output Shape              Param #
ConvModel                                [10, 32, 32]              --
├─Conv2d: 1-1                            [10, 32, 32]              130
Total params: 130
Trainable params: 130
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.04
Input size (MB): 0.01
Forward/backward pass size (MB): 0.08
Params size (MB): 0.00
Estimated Total Size (MB): 0.09

In [6]:
from tensorflow.keras import layers, Model   # Import Keras layers and Model class

image_size = 32                              # We will work with 32x32 pixel images

# 1) Define the input tensor for the model
inputs = layers.Input(                       # Functional API Input layer: start of the graph
    shape=(image_size,                       #   height  = 32
           image_size,                       #   width   = 32
           3)                                #   channels = 3 (RGB)
)                                            # Tensor shape: (None, 32, 32, 3); None = batch size

# 2) First “stage” tensor – just an alias of the inputs
d1 = inputs                                  # d1 now refers to the same tensor as `inputs`
                                            # No operation yet; shape is still (None, 32, 32, 3)


# 3) Convolution layer
d1 = layers.Conv2D(                          # 2-D convolution over the input image
        5,                                   #   out_channels / filters = 5
        kernel_size=(3, 3),                  #   filter size = 3x3
        padding='same',                      #   keep spatial size: 32x32 -> 32x32
        activation='gelu'                    #   apply GELU activation inside this layer
     )(d1)                                   # Immediately apply Conv2D to tensor d1
                                            # Output shape: (None, 32, 32, 5)

#   Parameter count for this Conv2D:
#   Formula: P = (N * M * D + 1) * K
#       N, M = kernel height, width = 3, 3
#       D    = input channels = 3
#       K    = number of filters = 5
#   So: P = (3 * 3 * 3 + 1) * 5 = (27 + 1) * 5 = 140 trainable parameters


# 4) Extra activation layer
d1 = layers.Activation('gelu')(d1)          # Apply GELU non-linearity again to the conv output
                                            # Note: Conv2D already used activation='gelu',
                                            # so this applies GELU twice. In practice you would
                                            # usually choose ONE of:
                                            #   - activation inside Conv2D, OR
                                            #   - a separate Activation layer.


# 5) Build the Keras model object
class_model = Model(                        # Create a Model that maps inputs → outputs
    inputs=inputs,                          #   model input tensor (images)
    outputs=d1                              #   final tensor after Conv2D + Activation
)                                           # This is a simple 1-layer CNN with GELU

# 6) Print the model architecture
class_model.summary()                       # Shows layers, output shapes, and parameter counts


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 5)      │           140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 5)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 140 (560.00 B)

 Trainable params: 140 (560.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
# If not installed in this runtime, first run:
# !pip install torchinfo -q

import torch.nn as nn
from torchinfo import summary


# ===============================
# 1. Define the model
# ===============================
class ConvGELUModel(nn.Module):
    """
    Simple CNN:
      - 1 Conv2d layer with 5 filters (3x3, 'same' padding)
      - GELU activation after the convolution
    """

    def __init__(self):
        # Initialize the parent nn.Module class
        super(ConvGELUModel, self).__init__()

        # Define a convolutional layer with 5 filters, 3x3 kernel size, and 'same' padding
        # in_channels  = 3    -> input has 3 channels (RGB image)
        # out_channels = 5    -> layer produces 5 feature maps
        # kernel_size  = (3,3)-> 3x3 convolution kernel
        # padding      = 'same' -> keep spatial size (H,W) the same (32x32 -> 32x32)
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=5,
            kernel_size=(3, 3),
            padding='same'
        )

        # Define the GELU activation function
        # This will be applied element-wise to the conv output
        self.gelu = nn.GELU()

    def forward(self, x):
        """
        Forward pass of the network.
        x: input tensor, shape (batch_size, 3, 32, 32)

        After conv1 + GELU:
        shape becomes (batch_size, 5, 32, 32)
        """
        # Apply the convolutional layer
        x = self.conv1(x)

        # Apply the GELU activation function
        x = self.gelu(x)

        # Return the final feature maps
        return x


# ===============================
# 2. Initialize the model
# ===============================
model = ConvGELUModel()   # create an instance of the model


# ===============================
# 3. Print model summary
# ===============================
# input_size is the size of a single input example (channels, height, width)
# Here: 3 channels (RGB), 32x32 pixels
summary(model, input_size=(3, 32, 32))   # (channels, height, width)


Layer (type:depth-idx)                   Output Shape              Param #
ConvGELUModel                            [5, 32, 32]               --
├─Conv2d: 1-1                            [5, 32, 32]               140
├─GELU: 1-2                              [5, 32, 32]               --
Total params: 140
Trainable params: 140
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.02
Input size (MB): 0.01
Forward/backward pass size (MB): 0.04
Params size (MB): 0.00
Estimated Total Size (MB): 0.05

In [8]:
from tensorflow.keras import layers, Model   # Import Keras layers and Model class

image_size = 32                              # We will use 32x32 pixel RGB images as input


# 1) Define the input tensor for the model
inputs = layers.Input(                       # Functional API Input layer: start of the graph
    shape=(image_size,                       #   height  = 32
           image_size,                       #   width   = 32
           3)                                #   channels = 3 (RGB)
)                                            # Tensor shape: (None, 32, 32, 3); None = batch size


# 2) First alias of the input
d1 = inputs                                  # d1 just points to the same tensor as `inputs`
                                             # No operation yet; still (None, 32, 32, 3)


# 3) Convolution + GELU activation
d1 = layers.Conv2D(                          # 2-D convolution layer
        filters=1,                           #   number of filters (output channels) = 1
        kernel_size=(3, 3),                  #   3x3 convolution kernel
        padding='same',                      #   keep H,W the same: 32x32 -> 32x32
        activation='gelu'                    #   apply GELU activation inside the layer
     )(d1)                                   # Immediately apply this Conv2D to tensor d1
                                             # Output shape: (None, 32, 32, 1)

#   Parameter count for this Conv2D:
#   P = (N * M * D + 1) * K
#       N, M = 3, 3 (kernel height, width)
#       D    = 3    (input channels)
#       K    = 1    (filters)
#   P = (3 * 3 * 3 + 1) * 1 = (27 + 1) * 1 = 28 trainable parameters


# 4) Batch Normalization
d1 = layers.BatchNormalization()(d1)         # Normalize activations over batch:
                                             #   y = (x - mean) / sqrt(var + eps) * gamma + beta
                                             # Trainable parameters per channel:
                                             #   gamma (scale) and beta (shift)
                                             # For 1 output channel → 2 trainable params (gamma, beta)
                                             # Plus 2 non-trainable (moving mean, moving variance)
                                             # Shape remains (None, 32, 32, 1)


# 5) Build the Keras model object
class_model = Model(                         # Create a Model mapping inputs → outputs
    inputs=inputs,                           #   model input tensor
    outputs=d1                               #   final tensor after Conv2D + BatchNorm
)                                            # This is a small CNN block: Conv + GELU + BN


# 6) Show the model architecture
class_model.summary()                        # Prints layers, output shapes, and parameter counts


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 1)      │            28 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 1)      │             4 │
│ (BatchNormalization)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32 (128.00 B)

 Trainable params: 30 (120.00 B)

 Non-trainable params: 2 (8.00 B)

In [12]:
!pip install torchinfo -q

import torch
import torch.nn as nn
from torchinfo import summary


class ConvGELUBNModel(nn.Module):
    """
    Conv + GELU + BatchNorm2d block.

    Input : (batch_size, 3, 32, 32)    # 3-channel 32x32 image (e.g. RGB)
    Output: (batch_size, 1, 32, 32)    # 1 feature map after conv+GELU+BN
    """

    def __init__(self):
        super(ConvGELUBNModel, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=1,
            kernel_size=(3, 3),
            padding=1          # padding=1 ≈ 'same' for 3x3
        )

        self.gelu = nn.GELU()
        self.batch_norm = nn.BatchNorm2d(num_features=1)

    def forward(self, x):
        x = self.conv1(x)       # (B, 3, 32, 32) -> (B, 1, 32, 32)
        x = self.gelu(x)        # non-linear activation
        x = self.batch_norm(x)  # normalize over batch & spatial dims
        return x


model = ConvGELUBNModel()

# IMPORTANT: include batch dimension N in input_size
summary(model, input_size=(1, 3, 32, 32))   # (batch, channels, height, width)


Layer (type:depth-idx)                   Output Shape              Param #
ConvGELUBNModel                          [1, 1, 32, 32]            --
├─Conv2d: 1-1                            [1, 1, 32, 32]            28
├─GELU: 1-2                              [1, 1, 32, 32]            --
├─BatchNorm2d: 1-3                       [1, 1, 32, 32]            2
Total params: 30
Trainable params: 30
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.03
Input size (MB): 0.01
Forward/backward pass size (MB): 0.02
Params size (MB): 0.00
Estimated Total Size (MB): 0.03

In [13]:
from tensorflow.keras import layers, Model

image_size = 32                              # Input images are 32×32 pixels

# ----- Input -----
inputs = layers.Input(                       # Functional API Input layer
    shape=(image_size, image_size, 3)        # (H, W, C) = (32, 32, 3) for RGB
)

d1 = inputs                                  # Start the block using the input tensor


# ----- Conv2D + GELU -----
d1 = layers.Conv2D(
        filters=5,                           # 5 output channels (feature maps)
        kernel_size=(3, 3),                  # 3×3 convolution kernel
        padding='same',                      # keep H,W = 32×32
        activation='gelu'                    # apply GELU inside this layer
    )(d1)                                    # apply conv to tensor d1
# Shape now: (None, 32, 32, 5)


# ----- Batch Normalization -----
d1 = layers.BatchNormalization()(d1)         # normalize each channel over the batch
# Shape remains: (None, 32, 32, 5)


# ----- MaxPooling -----
d1 = layers.MaxPool2D()(d1)                  # default pool_size=(2,2), stride=2
# Now H,W are halved: shape → (None, 16, 16, 5)


# ----- Build the model -----
class_model = Model(inputs=inputs, outputs=d1)
class_model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 5)      │           140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 5)      │            20 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 5)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160 (640.00 B)

 Trainable params: 150 (600.00 B)

 Non-trainable params: 10 (40.00 B)

In [4]:
# If torchinfo is not installed in this runtime, run once:
!pip install torchinfo -q

import torch
import torch.nn as nn
from torchinfo import summary


class ConvGELUBNMaxPoolModel(nn.Module):
    """
    PyTorch version of:
        Conv2D(5, 3x3, padding='same', activation='gelu')
        -> BatchNormalization
        -> MaxPool2D
    Input : (batch_size, 3, 32, 32)
    Output: (batch_size, 5, 16, 16)
    """

    def __init__(self):
        # Initialize parent nn.Module
        super(ConvGELUBNMaxPoolModel, self).__init__()

        # 1) Convolution layer
        #    Keras: Conv2D(5, kernel_size=(3,3), padding='same', activation='gelu')
        #
        # in_channels  = 3   → RGB image (3 channels)
        # out_channels = 5   → 5 filters → 5 output feature maps
        # kernel_size  = 3   → 3x3 kernel
        # padding      = 1   → for 3x3 kernel, padding=1 gives "same" spatial size:
        #                       input 32x32 → output 32x32
        self.conv = nn.Conv2d(
            in_channels=3,
            out_channels=5,
            kernel_size=3,
            padding=1
        )

        # 2) GELU activation
        #    Keras: activation='gelu'
        # Applies Gaussian Error Linear Unit element-wise.
        self.gelu = nn.GELU()

        # 3) Batch Normalization
        #    Keras: BatchNormalization()
        #
        # BatchNorm2d normalizes across (N, H, W) for each channel.
        # num_features = number of channels coming from conv layer = 5
        self.bn = nn.BatchNorm2d(num_features=5)

        # 4) Max pooling
        #    Keras: MaxPool2D()  (default pool_size=(2,2), strides=2)
        #
        # kernel_size = 2, stride = 2 → halves height and width:
        #   32x32 → 16x16
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        """
        Forward pass of the network.

        Input shape : (batch_size, 3, 32, 32)
        After conv   : (batch_size, 5, 32, 32)
        After GELU   : (batch_size, 5, 32, 32)
        After BN     : (batch_size, 5, 32, 32)
        After pool   : (batch_size, 5, 16, 16)
        """
        x = self.conv(x)   # Conv2d
        x = self.gelu(x)   # GELU activation
        x = self.bn(x)     # BatchNorm2d
        x = self.pool(x)   # MaxPool2d
        return x


# ----- create model instance -----
model = ConvGELUBNMaxPoolModel()

# ----- show summary (like model.summary() in Keras) -----
# torchinfo expects FULL 4D input size: (batch, channels, height, width)
summary(model, input_size=(1, 3, 32, 32))


Layer (type:depth-idx)                   Output Shape              Param #
ConvGELUBNMaxPoolModel                   [1, 5, 16, 16]            --
├─Conv2d: 1-1                            [1, 5, 32, 32]            140
├─GELU: 1-2                              [1, 5, 32, 32]            --
├─BatchNorm2d: 1-3                       [1, 5, 32, 32]            10
├─MaxPool2d: 1-4                         [1, 5, 16, 16]            --
Total params: 150
Trainable params: 150
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.14
Input size (MB): 0.01
Forward/backward pass size (MB): 0.08
Params size (MB): 0.00
Estimated Total Size (MB): 0.09